In [197]:
#Importing neccessary libraries
import pylast
import pickle
import calendar
import datetime as dt
import csv

In [198]:
#usernames = ['monkeyeason','bytrollinxon','schrammtheram','Pedro_Domingues','YannikTK']

In [199]:
# Function for creating a test pickle file containing usernames
# def createTestPickleFile(usernames):
#     db = {} 
#     db['username'] = usernames
#     dbfile = open('NamesPickle','ab') 
#     # source, destination 
#     pickle.dump(db, dbfile)                      
#     dbfile.close()  

In [200]:
#createTestPickleFile(usernames)

In [201]:
#Function for creating a test pickle file containing usernames 
#Will be modified for actual pickle file 
def loadPickleFile(name):
    dbfile = open(name, 'rb')      
    db = pickle.load(dbfile) 
    name_dict = {}
    for keys in db: 
        if (keys == 'username'): 
            names = db[keys] 
    dbfile.close()
    return names

In [216]:
# Kindly enter your private API_KEY, API_SECRET, username, and password
API_KEY = ""  
API_SECRET = ""
username = ""
password_hash = pylast.md5("")

In [203]:
network = pylast.LastFMNetwork(
    api_key=API_KEY,
    api_secret=API_SECRET,
    username=username,
    password_hash=password_hash,
)

In [204]:
#Kindly enter your own pickle file and if needed modify loadPickleFile
user_list = loadPickleFile('NamesPickle')

In [205]:
print(user_list)

['monkeyeason', 'bytrollinxon', 'schrammtheram', 'Pedro_Domingues', 'YannikTK']


In [206]:
# Function creates a user history csv file 
def write_history_csv(file_name, content):
    with open(file_name+'.csv','w+', newline='') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',')
        filewriter.writerow(['artist','artist_mbid','album','album_mbid','track','track_mbid','timestamp','datetime'])
        #for row in content:
        filewriter.writerows(content)

In [207]:
#Tag_heading to be inserted at top of tag csv
tag_heading = ["TrackName","ArtistName", "Playcount"]
for i in range(1,51):
    tag_heading.append("Tag"+str(i))
    tag_heading.append("TagWeight"+str(i))

#Function creates a csv file containing user tag for all tracks              
def write_tag_csv(file_name, content):
    with open(file_name+'_tag.csv','w+', newline='') as csvfile:
        filewriter = csv.writer(csvfile, delimiter=',')
        filewriter.writerow(tag_heading)
        filewriter.writerows(content)

In [208]:
#Function used to get tag details from tracks
def write_tag_details(username,artist_list):
    tag_content = []
    for item in artist_list:
        if(item[0] != None and item[1] != None):
            track = network.get_track(item[0],item[1])
            playcount = track.get_playcount()
            topTags = track.get_top_tags(limit=50)
            tag_row = [title,artist,playcount]
            for x in topTags:
                tag_row.append(x.item)
                tag_row.append(x.weight)
            tag_content.append(tag_row)
    write_tag_csv(username,tag_content)

In [209]:
#Function for finding user listening data for provided timespan
def generate_data_timespan(user_list,start,end,path):
    #Converting to time to UTC time zone
    utc_start = calendar.timegm(start.utctimetuple())
    utc_end = calendar.timegm(end.utctimetuple())
    
    for name in user_list:
        lastfm_user = network.get_user(name)
        tracks = lastfm_user.get_recent_tracks(time_from=utc_start, time_to=utc_end)
        user_tracks_history = []
        artist_list = []
        
        for track in tracks:
            track_mbid = None
            album_name = None
            artist_name = None
            album_mbid = None
            artist_mbid = None
            artist = None 
            title = None 
            album = None 

            if(track.track != None):
                track_mbid = track.track.get_mbid()
                title = track.track.get_title(properly_capitalized=True)
            if(track.album != None):
                album_name = track.album


            if(track.track.artist.name != None):
                artist_name = track.track.artist.name
                artist = network.get_artist(artist_name)
                artist_mbid = artist.get_mbid()

            if(album_name != None and artist_name != None ):
                album = network.get_album(artist_name,album_name)
                album_mbid = album.get_mbid()


            track_list = []
            track_list.append(artist_name) 
            track_list.append(artist_mbid)
            track_list.append(album_name)
            track_list.append(album_mbid)
            track_list.append(title)
            track_list.append(track_mbid)
            track_list.append(track.timestamp)
            track_list.append(track.playback_date)
            
            user_tracks_history.append(track_list)
            artist_list.append([artist_name,title])
            
        write_history_csv(path+'/'+name,user_tracks_history)
        write_tag_details(path+'/'+name,artist_list)

In [210]:
# sample_track = network.get_track('Broilers','Tanzt Du Noch Einmal Mit Mir?')
# count = sample_track.get_playcount()
# print(count)

In [211]:
start_pre_covid = dt.datetime(2019, 9, 1, 0, 0)
end_pre_covid = dt.datetime(2020, 3, 1, 0, 0)

start_during_covid = dt.datetime(2020, 3, 1, 0, 0)
end_during_covid = dt.datetime(2020, 9, 1, 0, 0)

In [213]:
generate_data_timespan(user_list,start_pre_covid,end_pre_covid,'./Pre')

In [214]:
generate_data_timespan(user_list,start_during_covid,end_during_covid,'./Dur')